<a href="https://colab.research.google.com/github/elyager/LLMs-from-scratch/blob/main/LLM_from_scratch_personal_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Requiriments

In [7]:
!pip install tiktoken
import torch
print("PyTorch version:", torch.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.1 MB/s eta 0:00:00
PyTorch version: 2.6.0+cu124


### Vocabulary

- **vocab_size:** es el tamaño del vocabulario en tokens únicos disponibles+extensiones. En nuestro caso dado por el tokenizador creado con BPE.
- **output_dim:** el número de dimensiones de cada token. Las dimensiones describen a una palabra o concepto. Más dimensiones capturan más detalles.
- **max_length:** es la máxima logitud de tokens por secuencia.
- **batch_size:** es cuantas secuencias (muestras de texto) tiene cada batch. Larger batch sizes can speed up training but might require more memory.
- **stride:** el tamaño de la zancada en tokens, cuantos tokens salta para la siguiente secuencia, esto determina que tanto se empalma una secuencia con otra. Ensure the model sees the context of each token multiple times during training, helping it learn better relationships between words.
- **shuffle:** determina si le da un orden aleatorio a las secuencias para que sea random en cada epoc. Prevents the model from learning patterns based on the order of data presentation. It helps generalize learning and avoid overfitting to a specific data order.
-**drop_last:** indica si se debe descartar el último batch cuando no cumple con el número de muestras establecido en max_lenght. Si se tiene un set de datos pequeño es mejor no hacer drop.
-**num_workers:** es el número de procesos, a mayor número más rápidez.
-**attention score:** determina qué tan "similar" es una pababra con la otra a través de dot product que es un tipo de similarity function. Mayor atention score mayor similitud entre los números.
--**attention weight:** es la versión normalizada a través de softmax de los attention scores.
-**context vector:** es un embedding vector pero que tiene todo el contexto del resto de input vectors. Se obtiene sumando todos los attention weights del inpute secuence.

# Chapter 1 & 2

### Load the text for training (our corpus)

In [19]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### BytePair Encoding (BPE)


In [8]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [21]:
tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
enc_text.append(tokenizer.eot_token)

# First 10 tokens from raw_text
first_10_token_ids = enc_text[:10]
decoded_tokens = [tokenizer.decode([token_id]) for token_id in first_10_token_ids]
delimited_tokens = ' |-|'.join(decoded_tokens)
print(delimited_tokens)
print(enc_text[:10])
print(f'\n Total of tokens: {len(enc_text)}')

I |-| H |-|AD |-| always |-| thought |-| Jack |-| G |-|is |-|burn |-| rather
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138]

 Total of tokens: 5146


### Dataset loader (creating tokenIDs for inputs and targets)

In [22]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        token_ids.append(tokenizer.eot_token)

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# dataset = GPTDatasetV1(raw_text, tokenizer, max_length=4, stride=1)
# print(dataset.input_ids)
# print(dataset.target_ids)

In [25]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

### Use DataLoader

In [28]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

# for batch in dataloader:
#     input, target = batch
#     print(input, target)

data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch) # input and target
second_batch = next(data_iter)
print(second_batch) # input and target

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
# First batch
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
print("Inputs:\n", tokenizer.decode(inputs[0].tolist()))
print("\nTargets:\n", tokenizer.decode(targets[0].tolist()))

# Second batch
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
print("Inputs:\n", tokenizer.decode(inputs[0].tolist()))
print("\nTargets:\n", tokenizer.decode(targets[0].tolist()))

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
Inputs:
 I HAD always

Targets:
  HAD always thought
Inputs:
 tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915]])

Ta

### Create our token embedding layer

In [33]:
vocab_size = 50257
output_dim = 3 # 256 is a more common starting point

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(token_embedding_layer.weight.shape)
print(token_embedding_layer.weight)

torch.Size([50257, 3])
Parameter containing:
tensor([[ 0.2253,  0.0226,  0.9778],
        [-0.5548,  1.7454, -0.3383],
        [-0.1281,  0.7334, -0.0365],
        ...,
        [ 1.4080,  0.5984,  1.2340],
        [ 1.0614,  0.6226, -2.2400],
        [ 0.9814,  0.5242,  0.3422]], requires_grad=True)


#### Load our dataset to get the inputs

In [34]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n",  inputs) # we take the first batch and  ignore the targets for now
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


### Create token embeddings

In [35]:
token_embeddings = token_embedding_layer(inputs)
# each token now has the assigned number of dimentions instead of being a single token ID
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
print(token_embeddings)

torch.Size([8, 4, 3])
tensor([[[ 0.2603, -0.8816,  0.1043],
         [ 0.2764, -0.4621, -0.7715],
         [ 0.8915, -1.0904, -0.2301],
         [-0.6681, -0.8020,  0.3033]],

        [[ 0.8063, -0.0582, -1.7538],
         [-0.3572,  0.4484,  1.3695],
         [-0.7448,  0.2903,  0.0090],
         [-1.9627, -0.5759,  2.8155]],

        [[ 1.5792, -0.2801,  0.4912],
         [ 0.6434,  0.0310,  0.0791],
         [-0.4761, -1.8900,  0.4606],
         [ 0.4095,  0.7004,  0.0515]],

        [[-1.2447, -0.6668,  0.5984],
         [ 0.4962, -1.0619,  1.0492],
         [-0.4329,  0.0821, -0.4267],
         [-0.4761, -1.8900,  0.4606]],

        [[-1.6483, -1.1164,  1.1943],
         [ 1.3296, -1.7755,  1.0300],
         [-1.0391, -0.5687, -1.3244],
         [ 0.4962, -1.0619,  1.0492]],

        [[ 0.8674,  0.3860,  1.9747],
         [-0.1526,  0.5415, -1.1297],
         [ 2.2568,  1.4764,  1.6068],
         [-0.2412,  0.1579, -1.8213]],

        [[-1.0868,  0.0871, -0.7187],
         [-1.022

### Create absolute positional embeddings

In [36]:
pos_embedding_layer = torch.nn.Embedding(max_length, output_dim)

# # [0, 1, 2, 3] "column" position is the position of each word on each sequence of 4 context_length
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# # uncomment & execute the following line to see how the embeddings look like
print(pos_embeddings)

torch.Size([4, 3])
tensor([[ 0.1183, -1.5899, -1.0680],
        [-1.4079, -2.6071, -0.4584],
        [-1.4400,  0.6619, -1.3333],
        [ 0.5600, -0.0192, -0.0558]], grad_fn=<EmbeddingBackward0>)


### Create input embeddings

In [37]:
input_embeddings = token_embeddings + pos_embeddings
print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')

# uncomment & execute the following line to see how the embeddings look like
print(input_embeddings)

torch.Size([8, 4, 3]) + torch.Size([4, 3]) = torch.Size([8, 4, 3])
tensor([[[ 3.7859e-01, -2.4715e+00, -9.6366e-01],
         [-1.1315e+00, -3.0692e+00, -1.2299e+00],
         [-5.4848e-01, -4.2846e-01, -1.5634e+00],
         [-1.0804e-01, -8.2112e-01,  2.4746e-01]],

        [[ 9.2461e-01, -1.6481e+00, -2.8217e+00],
         [-1.7651e+00, -2.1587e+00,  9.1108e-01],
         [-2.1848e+00,  9.5214e-01, -1.3243e+00],
         [-1.4027e+00, -5.9503e-01,  2.7597e+00]],

        [[ 1.6975e+00, -1.8701e+00, -5.7673e-01],
         [-7.6454e-01, -2.5761e+00, -3.7935e-01],
         [-1.9160e+00, -1.2282e+00, -8.7274e-01],
         [ 9.6952e-01,  6.8121e-01, -4.2917e-03]],

        [[-1.1264e+00, -2.2568e+00, -4.6952e-01],
         [-9.1170e-01, -3.6690e+00,  5.9074e-01],
         [-1.8728e+00,  7.4394e-01, -1.7600e+00],
         [ 8.3957e-02, -1.9092e+00,  4.0476e-01]],

        [[-1.5301e+00, -2.7064e+00,  1.2634e-01],
         [-7.8336e-02, -4.3826e+00,  5.7160e-01],
         [-2.4791e+00,  9

### What about more dimmension?

In [38]:
def more_more_more_dimensions(dim):
  vocab_size = 50257
  output_dim = dim

  token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
  token_embeddings = token_embedding_layer(inputs)
  print(token_embeddings.shape)
  print(token_embeddings)

  pos_embedding_layer = torch.nn.Embedding(max_length, output_dim)
  pos_embeddings = pos_embedding_layer(torch.arange(max_length))
  print(pos_embeddings.shape)
  print(pos_embeddings)


  input_embeddings = token_embeddings + pos_embeddings
  print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')
  print(input_embeddings)

more_more_more_dimensions(256)  #256 is a more common starting point

torch.Size([8, 4, 256])
tensor([[[ 1.6362, -0.9072,  1.1920,  ..., -2.6416, -0.6914, -0.3487],
         [-1.7458,  1.2970,  1.5771,  ...,  0.1099, -0.0311, -1.3486],
         [-0.9854, -0.8970, -0.7286,  ...,  0.8005, -0.0573,  2.1166],
         [ 0.0336,  2.2179,  1.0534,  ..., -0.4926,  1.3052,  1.7788]],

        [[-0.8723, -0.9800, -2.3819,  ..., -0.1328,  0.1684,  0.4709],
         [ 0.5318,  0.9099, -1.6467,  ...,  1.4716,  2.3577, -0.5886],
         [ 1.2070,  0.7688,  0.2764,  ..., -0.8464, -0.4858, -1.4624],
         [ 0.3716,  0.9414,  0.9643,  ..., -0.7349, -0.9102,  0.5917]],

        [[ 1.8858, -0.5925, -0.4191,  ...,  0.5168, -1.0823, -0.3816],
         [-0.8509,  0.7499, -0.9045,  ..., -1.6122,  0.3141,  1.6365],
         [-2.7887, -0.1300,  1.0576,  ...,  0.8582, -0.2795, -0.0770],
         [-0.8591,  0.8062,  0.2073,  ...,  1.6089, -0.0676, -1.3474]],

        ...,

        [[-1.6342,  0.2042, -1.4248,  ...,  0.5370,  0.4192, -1.5422],
         [-0.0173,  0.8601,  1.44

# Chapter 3

## Preparing data to work with

### Get input embeddings

In [39]:
def get_input_embeddings(my_raw_text):
  dataloader = create_dataloader_v1(my_raw_text, batch_size=1, max_length=6, stride=6, shuffle=False)

  data_iter = iter(dataloader)
  # First and only batch
  inputs, targets = next(data_iter)
  # print(inputs)
  # print(targets)  # ignore the targets

  vocab_size = 50257
  output_dim = 3

  token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
  token_embeddings = token_embedding_layer(inputs)
  context_length = 6
  pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
  pos_embeddings = pos_embedding_layer(torch.arange(context_length))

  input_embeddings = token_embeddings + pos_embeddings
  # print(f'{token_embeddings.shape} + {pos_embeddings.shape} = {input_embeddings.shape}')
  # print(input_embeddings)
  return input_embeddings

In [40]:
my_raw_text = "Your journey starts with one step"
small_input_embeddings = get_input_embeddings(my_raw_text)[0]
print(small_input_embeddings)

tensor([[ 2.2861, -0.6132, -1.6731],
        [-0.4499, -2.7230,  0.6959],
        [-0.8747, -1.1428, -2.6916],
        [ 0.3154,  0.2821, -1.1845],
        [ 0.7820,  0.5111, -1.1167],
        [-1.3217,  1.8591,  2.0771]], grad_fn=<SelectBackward0>)


In [ ]:
# tensor([
#     [ 0.4113,  1.3397, -1.2234], Your    (x^1)
#     [-1.8881, -0.0679, -1.1267], journey (x^2)
#     [-0.2323, -2.2089, -1.6685], starts  (x^3)
#     [ 0.5615,  1.2698,  2.5768], with    (x^4)
#     [-0.9290, -0.0227,  0.6467], one     (x^5)
#     [ 0.5691, -2.0627, -3.2411]  step    (x^6)
# ])

### Forcing values to fit between 0 and 1

In [41]:
import decimal
min_val = small_input_embeddings.min()
max_val = small_input_embeddings.max()
scaled_embeddings = (small_input_embeddings - min_val) / (max_val - min_val)
rounded_embeddings = torch.round(scaled_embeddings * 100) / 100
print(rounded_embeddings)

tensor([[1.0000, 0.4200, 0.2100],
        [0.4500, 0.0000, 0.6800],
        [0.3700, 0.3200, 0.0100],
        [0.6100, 0.6000, 0.3100],
        [0.7000, 0.6500, 0.3200],
        [0.2800, 0.9100, 0.9600]], grad_fn=<DivBackward0>)


## Simple self-attention

### Step 1 - Compute unormalized attention scores

In [42]:
query = rounded_embeddings[1]  # 2nd input token is the query)
# just allocate a tensor in memory with 6 spaces
attn_scores_2 = torch.empty(rounded_embeddings.shape[0])

#fill the tensor with the dot products which multiply and sum
for i, x_i in enumerate(rounded_embeddings):
    print(f'dot product of {x_i} against journey {query}')
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

dot product of tensor([1.0000, 0.4200, 0.2100], grad_fn=<UnbindBackward0>) against journey tensor([0.4500, 0.0000, 0.6800], grad_fn=<SelectBackward0>)
dot product of tensor([0.4500, 0.0000, 0.6800], grad_fn=<UnbindBackward0>) against journey tensor([0.4500, 0.0000, 0.6800], grad_fn=<SelectBackward0>)
dot product of tensor([0.3700, 0.3200, 0.0100], grad_fn=<UnbindBackward0>) against journey tensor([0.4500, 0.0000, 0.6800], grad_fn=<SelectBackward0>)
dot product of tensor([0.6100, 0.6000, 0.3100], grad_fn=<UnbindBackward0>) against journey tensor([0.4500, 0.0000, 0.6800], grad_fn=<SelectBackward0>)
dot product of tensor([0.7000, 0.6500, 0.3200], grad_fn=<UnbindBackward0>) against journey tensor([0.4500, 0.0000, 0.6800], grad_fn=<SelectBackward0>)
dot product of tensor([0.2800, 0.9100, 0.9600], grad_fn=<UnbindBackward0>) against journey tensor([0.4500, 0.0000, 0.6800], grad_fn=<SelectBackward0>)
tensor([0.5928, 0.6649, 0.1733, 0.4853, 0.5326, 0.7788], grad_fn=<CopySlices>)


### Step 2 - Normalize the attenton scores to sum up to 1

In [43]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1731, 0.1861, 0.1138, 0.1555, 0.1630, 0.2085],
       grad_fn=<DivBackward0>)
Sum: tensor(1.0000, grad_fn=<SumBackward0>)


In [66]:
# Fooling around with dimensions on vectors
my_tensor = torch.ones(2,3,4) # I always start with the last dimension which is [-1]
print(my_tensor)
print(my_tensor.shape)
print(my_tensor.shape[-1]) # last dimension
print(my_tensor.shape[0]) # first dimenson
print(my_tensor.shape[1])
print(my_tensor.shape[2]) # same as [-1]
# print(my_tensor.shape[3]) # IndexError

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
torch.Size([2, 3, 4])
4
2
3
4


In [72]:
# using pytorch softmax fucntion
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum()) #100%


Attention weights: tensor([0.1731, 0.1861, 0.1138, 0.1555, 0.1630, 0.2085],
       grad_fn=<SoftmaxBackward0>)
Sum: tensor(1., grad_fn=<SumBackward0>)


### Step 3 - Compute the context vector $z^{(2)}$

In [69]:
query = rounded_embeddings[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
attn_weights_sum = 0
for i,x_i in enumerate(rounded_embeddings):
    print(f'{attn_weights_2[i]} * {x_i}')
    context_vec_2 += attn_weights_2[i]*x_i
    attn_weights_sum += attn_weights_2[i]

print(attn_weights_sum)
print(context_vec_2)

0.1731238067150116 * tensor([1.0000, 0.4200, 0.2100], grad_fn=<UnbindBackward0>)
0.18606702983379364 * tensor([0.4500, 0.0000, 0.6800], grad_fn=<UnbindBackward0>)
0.1138073280453682 * tensor([0.3700, 0.3200, 0.0100], grad_fn=<UnbindBackward0>)
0.15547841787338257 * tensor([0.6100, 0.6000, 0.3100], grad_fn=<UnbindBackward0>)
0.16300925612449646 * tensor([0.7000, 0.6500, 0.3200], grad_fn=<UnbindBackward0>)
0.20851415395736694 * tensor([0.2800, 0.9100, 0.9600], grad_fn=<UnbindBackward0>)
tensor(1.0000, grad_fn=<AddBackward0>)
tensor([0.5663, 0.4981, 0.4646], grad_fn=<AddBackward0>)


### Get All attention weights

In [73]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(rounded_embeddings):
    for j, x_j in enumerate(rounded_embeddings):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[1.2205, 0.5928, 0.5065, 0.9271, 1.0402, 0.8638],
        [0.5928, 0.6649, 0.1733, 0.4853, 0.5326, 0.7788],
        [0.5065, 0.1733, 0.2394, 0.4208, 0.4702, 0.4044],
        [0.9271, 0.4853, 0.4208, 0.8282, 0.9162, 1.0144],
        [1.0402, 0.5326, 0.4702, 0.9162, 1.0149, 1.0947],
        [0.8638, 0.7788, 0.4044, 1.0144, 1.0947, 1.8281]],
       grad_fn=<CopySlices>)


We can achive the same but more efficiently via matrix multiplication

In [74]:
# Using matrix transpose (remember is row * column)
print(rounded_embeddings.shape)
print(rounded_embeddings.T.shape)
print('-------------------')
print(rounded_embeddings)
print('-------------------')
print(rounded_embeddings.T)

torch.Size([6, 3])
torch.Size([3, 6])
-------------------
tensor([[1.0000, 0.4200, 0.2100],
        [0.4500, 0.0000, 0.6800],
        [0.3700, 0.3200, 0.0100],
        [0.6100, 0.6000, 0.3100],
        [0.7000, 0.6500, 0.3200],
        [0.2800, 0.9100, 0.9600]], grad_fn=<DivBackward0>)
-------------------
tensor([[1.0000, 0.4500, 0.3700, 0.6100, 0.7000, 0.2800],
        [0.4200, 0.0000, 0.3200, 0.6000, 0.6500, 0.9100],
        [0.2100, 0.6800, 0.0100, 0.3100, 0.3200, 0.9600]],
       grad_fn=<PermuteBackward0>)


In [75]:
attn_scores = rounded_embeddings @ rounded_embeddings.T
print(attn_scores)

tensor([[1.2205, 0.5928, 0.5065, 0.9271, 1.0402, 0.8638],
        [0.5928, 0.6649, 0.1733, 0.4853, 0.5326, 0.7788],
        [0.5065, 0.1733, 0.2394, 0.4208, 0.4702, 0.4044],
        [0.9271, 0.4853, 0.4208, 0.8282, 0.9162, 1.0144],
        [1.0402, 0.5326, 0.4702, 0.9162, 1.0149, 1.0947],
        [0.8638, 0.7788, 0.4044, 1.0144, 1.0947, 1.8281]],
       grad_fn=<MmBackward0>)


Apply softmax

In [76]:
# dim=-1 is to apply softmax to the last dimenison, in this case rows
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2323, 0.1240, 0.1138, 0.1733, 0.1940, 0.1626],
        [0.1731, 0.1861, 0.1138, 0.1555, 0.1630, 0.2085],
        [0.1898, 0.1360, 0.1453, 0.1743, 0.1831, 0.1714],
        [0.1911, 0.1229, 0.1152, 0.1731, 0.1891, 0.2086],
        [0.1968, 0.1184, 0.1113, 0.1738, 0.1919, 0.2078],
        [0.1317, 0.1209, 0.0832, 0.1531, 0.1659, 0.3453]],
       grad_fn=<SoftmaxBackward0>)


In [77]:
row_0_sum = sum([0.1403, 0.1365, 0.1915, 0.1552, 0.1659, 0.2106])
print(row_0_sum)
print("All row sums:", attn_weights.sum(dim=-1))

1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)


### Compute All context vectors

In [78]:
all_context_vecs = attn_weights @ rounded_embeddings
print(all_context_vecs)
print("Previous 2nd context vector:", context_vec_2)

tensor([[0.6173, 0.5120, 0.4062],
        [0.5663, 0.4981, 0.4646],
        [0.5873, 0.5058, 0.4110],
        [0.5854, 0.5337, 0.4393],
        [0.5898, 0.5364, 0.4378],
        [0.5230, 0.5958, 0.5428]], grad_fn=<MmBackward0>)
Previous 2nd context vector: tensor([0.5663, 0.4981, 0.4646], grad_fn=<AddBackward0>)


## Self Attention

**Weight parameters** are learned coefficients that define the network connections, while **attention weights** are dynamic, context-specific values.

In [79]:
x_2 = rounded_embeddings[1] # second input element
d_in = rounded_embeddings.shape[1] #depends on the embeddings dimension the input embedding size, dim=3
d_out = 2 # the output embedding size, dim=2

torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

print(W_query)
print(W_key)
print(W_value)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])
Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [80]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

# calculate keys and values vector for all inputs
keys = rounded_embeddings @ W_key
values = rounded_embeddings @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

tensor([0.1836, 0.8217], grad_fn=<SqueezeBackward4>)
keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [81]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(0.4724, grad_fn=<DotBackward0>)


In [82]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([0.5049, 0.4724, 0.2485, 0.6380, 0.6856, 1.2021],
       grad_fn=<SqueezeBackward4>)


The difference to earlier is that we now scale the attention scores by dividing them by the square root of the embedding dimension,  𝑑𝑘‾‾‾√  (i.e., d_k**0.5):

Imagine you have two vectors, and their dot product results in a large value. When this large value is passed through the softmax function, it might dominate the probabilities, making the attention mechanism less sensitive to other relevant parts of the input. Scaling helps to mitigate this issue by preventing any single dot product from becoming overly influential.

In [ ]:
d_k = keys.shape[1] # dimension of keys
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1448, 0.1576, 0.1573, 0.1993, 0.1977, 0.1433],
       grad_fn=<SoftmaxBackward0>)


In [83]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.2555, 0.6958], grad_fn=<SqueezeBackward4>)


### Compact SelfAttention Class

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- We can streamline the implementation above using PyTorch's Linear layers instead of torch random, which are equivalent to a matrix multiplication if we disable the bias units
- Another big advantage of using `nn.Linear` over our manual `nn.Parameter(torch.rand(...)` approach is that `nn.Linear` has a preferred weight initialization scheme, which leads to more stable model training

In [84]:
import torch.nn as nn

In [85]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

In [86]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(rounded_embeddings))

tensor([[-0.0079,  0.1668],
        [-0.0053,  0.1702],
        [-0.0075,  0.1675],
        [-0.0079,  0.1668],
        [-0.0081,  0.1665],
        [-0.0073,  0.1675]], grad_fn=<MmBackward0>)


### Hiding futer words with causal attention (one step back)

In [87]:
# Reuse data from previous section
queries = sa_v2.W_query(rounded_embeddings)
keys = sa_v2.W_key(rounded_embeddings)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1694, 0.1863, 0.1518, 0.1606, 0.1617, 0.1702],
        [0.1721, 0.1817, 0.1576, 0.1624, 0.1632, 0.1630],
        [0.1672, 0.1752, 0.1594, 0.1640, 0.1644, 0.1698],
        [0.1695, 0.1866, 0.1516, 0.1605, 0.1616, 0.1702],
        [0.1696, 0.1885, 0.1502, 0.1599, 0.1611, 0.1706],
        [0.1732, 0.1988, 0.1449, 0.1571, 0.1587, 0.1672]],
       grad_fn=<SoftmaxBackward0>)


Applying negative infinity effectively zeros out the probabilities for these future tokens in the subsequent softmax calculation.

In [88]:
context_length = attn_scores.shape[-1]
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.1598,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1269, 0.2032,   -inf,   -inf,   -inf,   -inf],
        [0.0697, 0.1352, 0.0021,   -inf,   -inf,   -inf],
        [0.1624, 0.2988, 0.0044, 0.0857,   -inf,   -inf],
        [0.1770, 0.3265, 0.0048, 0.0939, 0.1040,   -inf],
        [0.2582, 0.4533, 0.0061, 0.1200, 0.1349, 0.2085]],
       grad_fn=<MaskedFillBackward0>)


In [89]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4865, 0.5135, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3490, 0.3177, 0.0000, 0.0000, 0.0000],
        [0.2536, 0.2793, 0.2268, 0.2402, 0.0000, 0.0000],
        [0.2045, 0.2273, 0.1811, 0.1929, 0.1942, 0.0000],
        [0.1732, 0.1988, 0.1449, 0.1571, 0.1587, 0.1672]],
       grad_fn=<SoftmaxBackward0>)


### Masking additional attention weights with dropout

In [90]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))
print(dropout(attn_weights))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])
tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6354, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5586, 0.0000, 0.4805, 0.0000, 0.0000],
        [0.0000, 0.4546, 0.3621, 0.3857, 0.3885, 0.0000],
        [0.3464, 0.3976, 0.0000, 0.0000, 0.3175, 0.3345]],
       grad_fn=<MulBackward0>)


### Causal Attention Class

In [ ]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        print(b)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [91]:
# torch.manual_seed(123)

# batch = torch.stack((rounded_embeddings, rounded_embeddings), dim=0)
# print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

# context_length = batch.shape[1]
# ca = CausalAttention(d_in, d_out, context_lenght, 0.0)

# context_vecs = ca(batch)

# print(context_vecs)
# print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 6, 3])


NameError: name 'CausalAttention' is not defined

## Self Attention multi-head

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape #b is for batches
        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        print('--------------------')
        print(f'{keys.shape} vs {keys.transpose(1,2).shape}')
        print('--------------------')
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

--------------------
torch.Size([2, 6, 2, 1]) vs torch.Size([2, 2, 6, 1])
--------------------
tensor([[[0.2699, 0.4130],
         [0.2399, 0.4178],
         [0.2289, 0.4308],
         [0.2306, 0.4467],
         [0.2335, 0.4329],
         [0.2335, 0.4581]],

        [[0.2699, 0.4130],
         [0.2399, 0.4178],
         [0.2289, 0.4308],
         [0.2306, 0.4467],
         [0.2335, 0.4329],
         [0.2335, 0.4581]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
